# Tarea 2 
 
Modelo de tasas Hull-White con simulaciones de Montecarlo

Integrantes:
- Joaquin Garay
- Lucas Ribas

## Inicialización funciones previas

In [1]:
from scipy.interpolate import interp1d
from scipy.stats import norm
import plotly.express as px
import pandas as pd
import numpy as np
import math

Se obtienen los valores de la curva cero cupón:

In [2]:
curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [3]:
curva['t'] = curva['plazo'] / 365.0

### Funciones curva cero cupón.

Se definen las funciones relacionadas a la curva cero cupon.
- `zrate`: Interpola los valores de la curva cero cupón con interpolación cúbica.
- `dzrate`: Primera derivada de la curva cero cupón. Diferencia finita forward.
- `dz2rate`: Segunda derivada de la curva cero cupón. Diferencia finita forward.
- `fwd`: Valor de la tasa forward determinada por $\displaystyle f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$ 
- `dfwd`: Primera derivada de la tasa forward determinada por $\displaystyle \frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$


In [4]:
def zrate(t: float):
    dfcurva = interp1d(curva['t'],
                  curva['tasa'],
                  kind='cubic',
                  fill_value="extrapolate")
    return dfcurva(t)

def dzrate(t: float) -> float:
    delta = .0001
    return (zrate(t + delta) - zrate(t)) / delta

def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [5]:
t = 2
print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')
print(f'fwd({t:.2f}) = {fwd(t):.4%}')
print(f'dfwd({t:.2f}) = {dfwd(t):.4%}')

zrate(2.00) = 0.0568%
dzrate(2.00) = -0.0066%
d2zrate(2.00) = 0.0584%
fwd(2.00) = 0.0436%
dfwd(2.00) = 0.1036%


In [6]:
# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(0)
print(f'Tasa instantánea: {r0:.4%}')

Tasa instantánea: 0.0788%


### Funciones bono cero cupón

Se definen además las funciones relacionadas a los bonos cero cupón. La fórmula para la valoración de bonos cero cupón en el modelo de Hull - White es la siguiente:

$$
\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right]
\end{equation}
$$

$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp(-\gamma^* (T-t))\right]
\end{equation}
$$

$$
\begin{equation}
A\left(t,T\right)=\log\frac{Z\left(r_0,0,T\right)}{Z\left(r_0,0,t\right)}+B\left(t,T\right)f\left(0,t\right)-\frac{\sigma^2}{4\gamma^*}B\left(t,T\right)^2\left[1-\exp\left(-2\gamma^* t\right)\right]
\end{equation}
$$

In [7]:
sigma = .015
gamma = .5

In [8]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    df_T = math.exp(-zrate(T) * T)
    df_t = math.exp(-zrate(t) * t)
    c1 = math.log(df_T / df_t)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))

    return c1 + c2 - c3

def zero_hw(r: float,
            gamma: float,
            sigma: float,
            zrate: float,
            fwd: float,
            t: float,
            T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

### Función theta

Se inicializa la funcion $\theta_t$ en medida forward `theta_fwd_measure` y en medida ajustada por riesgo `theta_rf_measure`.

$$
\overline{\theta_t}=\theta_t-B\left(t,T\right)\sigma^2
$$

donde $\overline{\theta_t}$ es la función $\theta$ en la medida forward y $\theta_t$ en medida ajustada por riesgo. $\displaystyle \theta_t=\frac{\partial f\left(0,t\right)}{\partial t}+\gamma^*f\left(0,t\right)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp\left(-2\gamma^* t\right)\right]$



In [9]:
def theta_rf_measure(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

def theta_fwd_measure(t: float, T:float) -> float: 
    return theta_rf_measure(t) - b_hw(gamma, t, T)*sigma**2

### Funciones call y put para bono cero cupón.

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$

$$\displaystyle d_1=\frac{\log\left(\frac{Z(r_0,0,T_B)}{K\cdot Z(r_0,0,T_O)}\right)+\frac{1}{2}S_Z(T_O)^2}{S_Z(T_O)} \quad \text{y} \quad d_2=d_1-S_Z(T_O)$$

La volatilidad
$$\displaystyle S_Z(T_O)=B(T_O,T_B)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp(-2\gamma^* T_O)\right)}$$


Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$. 

In [10]:
def call_hw(r0: float, To: float, Tb: float, K: float, notional:float, t = 0):
    
    # Se normaliza strike para ser consistente con la formula. Precio strike por unidad de bono.
    K = K/notional
    
    # Se definen los precios de los bonos zero coupon para Tb (Tenor bono) y To (Tenor opción)
    Z_Tb = zero_hw(r0, gamma, sigma, zrate, fwd, t, Tb)
    Z_To = zero_hw(r0, gamma, sigma, zrate, fwd, t, To)
    
    # Se definen las expresiones características del modelo
    Sz = b_hw(gamma, To, Tb) * (sigma**2/(2*gamma))**0.5 * (1-np.exp(-2*gamma*To))**0.5
    d1 = ( np.log(Z_Tb/(K*Z_To)) + 0.5*Sz**2 ) / Sz
    d2 = d1 - Sz
    
    call = Z_Tb*norm.cdf(d1)-Z_To*K*norm.cdf(d2)
    
    return notional*call

def put_hw(r0: float, To: float, Tb: float, K: float, notional:float, t = 0):
    
    # Se normaliza strike para ser consistente con la formula. Precio strike por unidad de bono.
    K = K/notional
    
    # Se definen los precios de los bonos zero coupon para Tb (Tenor bono) y To (Tenor opción)
    Z_Tb = zero_hw(r0, gamma, sigma, zrate, fwd, t, Tb)
    Z_To = zero_hw(r0, gamma, sigma, zrate, fwd, t, To)
    
    # Se definen las expresiones características del modelo
    Sz = b_hw(gamma, To, Tb) * (sigma**2/(2*gamma))**0.5 * (1-np.exp(-2*gamma*To))**0.5
    d1 = ( np.log(Z_Tb/(K*Z_To)) + 0.5*Sz**2 ) / Sz
    d2 = d1 - Sz
    
    put = -1*(Z_Tb*norm.cdf(-d1)-Z_To*K*norm.cdf(-d2))
    
    return notional*put

## Simulación de Monte Carlo medida Forward para Calls y Puts

En la medida forward, el proceso estocástico de la tasa corta se lee como

$$
dr_t=\left(\bar{\theta_t}-\gamma^* r_t\right)dt+\sigma dX_t
$$


Y una posible discretización del proceso corresponde a 
$$ r_{t+1} = r_{t} + (\bar{\theta_t} - \gamma^* r_t)\cdot dt + \sigma \sqrt{dt} \cdot Shock_t $$
donde $Shock_t$ es una variable aleatoria que distribuye normal (0,1).

De manera adicional, se aplica la técnica de reducción de varianza de variables antitéticas, con tal de lograr mejor precisión en pocas simulaciones.

In [11]:
def option_value_MC_fwd(gamma, sigma, theta_fwd_measure, zrate, To, Tb, r0, K, notional, 
                        call_put_flag, num_sim, num_steps, seed = None):
    """
    Valora opcion call y put sobre bonos cero cupon con el modelo de Hull White con simulaciones de Monte Carlo
    en la medida forward.
    """
    # Call Put flag
    eps = call_put_flag
    
    # Pasos temporales
    dt = 1 / 264.0
    num_steps += 1
    
    # Se precalculan los números aleatorios
    random_numbers = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            random_numbers[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    for i in range(0, num_steps):
        theta_array[i] = theta_fwd_measure(i * dt, To)
    
    # Precalculo de constantes para la simulacion
    df = math.exp(-To * float(zrate(To)))
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    
    # Inicializacion acumuladores
    acum_pricing = 0
    acum_sim = 0
    
    # Se comienza el loop de las simulaciones
    for i in range(0, num_sim):
        
        # Se recorre la trayectoria y se guarda la tasa con que termina el path.
        # Se simulan dos tractorias a la vez con tal de usar técnica antitética de reducción de varianza.
        r1 = r0
        r2 = r0
        for j in range(1, num_steps):
            r1 = r1 + theta_array[j - 1] * dt - gamma_dt * r1 + sqdt_sigma * random_numbers[i][j - 1]
            r2 = r2 + theta_array[j - 1] * dt - gamma_dt * r2 - sqdt_sigma * random_numbers[i][j - 1]
            
        r_last1 = r1   
        r_last2 = r2
        
        # Se calcula el precio del bono cero cupon con
        price_bzc1 = notional*zero_hw(r_last1, gamma, sigma, zrate, fwd, To, Tb)
        price_bzc2 = notional*zero_hw(r_last2, gamma, sigma, zrate, fwd, To, Tb)
        
        # Se determina el payoff de la opcion
        payoff1 = max(eps*(price_bzc1 - K), 0)
        payoff2 = max(eps*(price_bzc2 - K), 0)
        
        # Se trae a valor presente el payoff
        value1 = df * payoff1
        value2 = df * payoff2
        
        # Se añade al acumulador
        acum_pricing += (value1 + value2)/2
        acum_sim += 1
        
    option_value = acum_pricing / acum_sim
    
    return option_value

### Cálculo Call

In [22]:
# Test función option_value_MC_fwd para opción Call
num_sim = 1000
num_steps = 264
seed = 2
K = 98
r0 = zrate(0)
To = 1
Tb = 2
notional = 100
call_put_flag = 1

call_MC = option_value_MC_fwd(gamma, sigma, theta_fwd_measure, zrate, To, Tb, r0, K, notional, 
                              call_put_flag, num_sim, num_steps, seed)
call_HW_closed = call_hw(r0, To, Tb, K, notional, t = 0)

In [23]:
print(f'Valor call con simulación MC: {call_MC:.8}')
print(f"Valor call con fórmula cerrada: {call_HW_closed:.8}")
diff = np.abs(call_MC - call_HW_closed)
print(f"Error en la valorización: {diff:.8%}")

Valor call con simulación MC: 1.9588805
Valor call con fórmula cerrada: 1.9612536
Error en la valorización: 0.23731002%


### Tests Call

In [24]:
import sys
sys.path.insert(1, '../modules')
import hull_white as hw

In [25]:
hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=K/100,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma) * 100

1.9612536434544858

### Cálculo Put

In [26]:
# Test función option_value_MC_fwd para opción Put
num_sim = 1000
num_steps = 264
seed = 2
K = 103
r0 = zrate(0)
To = 1
Tb = 2
notional = 100
call_put_flag = -1

put_MC = option_value_MC_fwd(gamma, sigma, theta_fwd_measure, zrate, To, Tb, r0, K, notional, 
                             call_put_flag, num_sim, num_steps, seed)
put_HW_closed = put_hw(r0, To, Tb, K, notional, t = 0)

In [27]:
print(f'Valor put con simulación MC: {put_MC:.8}')
print(f"Valor put con fórmula cerrada: {put_HW_closed:.8}")
diff = np.abs(put_MC - put_HW_closed)
print(f"Error en la valorización: {diff:.8%}")

Valor put con simulación MC: 3.0431627
Valor put con fórmula cerrada: 3.0413109
Error en la valorización: 0.18518109%


In [28]:
hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=K/100,
    to=To,
    tb=Tb,
    r0=r0,
    zo=math.exp(-zrate(To) * To),
    zb=math.exp(-zrate(Tb) * Tb),
    gamma=gamma,
    sigma=sigma) * 100

3.041310868796987

**AD:** hasta ahora muy claro todo. Me hubiera gustado que hubieran testeado con más valores de strikes, vencimientos de opción y vencimientos de bono cero cupón.

## Simulación de Montecarlo bono zero cupón medida Riesgo Neutral

Para valorizar un bono cero cupón con simulaciones de Montecarlo y compararlo con los precios de mercado, se debe usar una simulación en medida neutral al riesgo, por lo que se define nuevamente una funcion Monte Carlo.

In [18]:
def bond_value_MC_rf(gamma, sigma, theta_rf_measure, r0, notional, num_sim, num_steps, seed = None):
    """
    Valora bono cero cupon con el modelo de Hull White con simulaciones de Monte Carlo
    en la medida riesgo neutral.
    """
    # Pasos temporales
    dt = 1 / 264.0
    num_steps += 1
    
    # Se precalculan los números aleatorios
    random_numbers = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            random_numbers[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    for i in range(0, num_steps):
        theta_array[i] = theta_rf_measure(i * dt)
    
    # Precalculo de constantes para la simulacion
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    
    # Inicializacion acumuladores
    acum_sim = 0
    acum_df = 0
    
    # Se comienza el loop de las simulaciones
    for i in range(0, num_sim):
        
        sim1 = np.zeros(num_steps)
        sim2 = np.zeros(num_steps)
        sim1[0] = r0
        sim2[0] = r0
        r1 = r0
        r2 = r0
        
        # Se recorre y se guarda la trayectoria de tasas.
        for j in range(1, num_steps):
            r1 = r1 + theta_array[j - 1] * dt - gamma_dt * r1 + sqdt_sigma * random_numbers[i][j - 1]
            r2 = r2 + theta_array[j - 1] * dt - gamma_dt * r2 - sqdt_sigma * random_numbers[i][j - 1]
            sim1[j] = r1
            sim2[j] = r2

        acum_df += 0.5*(math.exp(-dt * np.sum(sim1)) + math.exp(-dt * np.sum(sim2)))
        acum_sim += 1

    price_bzc = acum_df / acum_sim
        
    return price_bzc*notional

In [19]:
num_steps = 263
num_sim = 1000
notional = 1

test_bzc = bond_value_MC_rf(gamma, sigma, theta_rf_measure, r0, notional, num_sim, num_steps, seed)

In [20]:
print(f'Valor bzc con simulación MC: {test_bzc:.4%}')
z_curva = math.exp(-zrate(1) * (1))
print(f"Valor bzc de mercado: {z_curva:.4%}")
diff = np.abs(test_bzc - z_curva)
print(f"Error en la valorización: {diff:.4%}")

Valor bzc con simulación MC: 99.9266%
Valor bzc de mercado: 99.9298%
Error en la valorización: 0.0032%


**AD:** ídem anterior. Más plazos testeados.